# Data Collection & Aggregation

_Authors: Colleen Kenney, Jerel Novick_

---


## Contents

[1. Collecting Individual Businesses in each city](#1.-Collecting-Individual-Businesses-in-each-city) <br>
[2. Collecting Affluence Data for each city](#2.-Collecting-Affluence-Data-for-each-city) <br>
[3. Aggregating businesses by city](#3.-Aggregating-businesses-by-city)<br>

### 1. Collecting Individual Businesses in each city

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from yelpapi import YelpAPI
import time
import regex as re
from IPython.display import clear_output

In [2]:
# https://gist.github.com/rugbyprof/76575b470b6772ce8fa0c49e23931d97

states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado",
          "CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho",
          "IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana",
          "ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi",
          "MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey",
          "NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma",
          "OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota",
          "TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington",
          "WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_largest_cities_of_U.S.'
                           +'_states_and_territories_by_population').text

soup = BeautifulSoup(url,'lxml')

cities = []
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    city = {}
    data = items.find_all(['th','td'])
    try:
        state = data[0].a.text
        most_populous_city = data[2].a.text
    except IndexError:pass
    city['state'] = state
    city['most_populous_city'] = most_populous_city
    city['city,state'] = most_populous_city + ", " + state 
    cities.append(city)
    
city_df = pd.DataFrame(cities)
city_df = city_df[city_df['state'].isin(list(states.values()))]
city_df['abbrev_state'] = list(states.keys())
city_df.to_csv('../data/cities.csv',index=False)

In [4]:
cities = list(city_df['city,state'])

In [5]:
# https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [6]:
## your yelp api key can be retrieved at https://www.yelp.com/fusion
## this code will not work unless you fill in your_key with an api key

yelp_api = YelpAPI(your_key)

def gather_data(locations,category):

    rows = []
    for location in range(len(locations)):

        counter = 1
        category_in_state = []

        for i in range(10):
            response = yelp_api.search_query(categories = category,location=locations[location],
                                                 sort_by='review_count', limit=50, offset=counter)
            counter += 50

            for i in range(len(response['businesses'])):
                row_dict = {}
                row_dict['name'] = response['businesses'][i].get('name')
                row_dict['state'] = response['businesses'][i].get('location')['state']
                row_dict['city'] = response['businesses'][i].get('location')['city']
                row_dict['price'] = response['businesses'][i].get('price')
                row_dict['review_count'] = response['businesses'][i].get('review_count')
                row_dict['rating'] = response['businesses'][i].get('rating')
                row_dict['category'] = category
                category_in_state.append(row_dict)
                rows.append(row_dict)
        
        time.sleep(3)
        update_progress((location+1)/len(locations))
        print(f"{len(category_in_state)} Total {category} collected in {locations[location]}")
        
    df = pd.DataFrame(rows).drop_duplicates()
    df = df[df['state'].isin(list(states.keys()))].dropna(subset=['price'])
    df.to_csv(f"../data/individual_{category}.csv",index=False)
    
    return df

```restaurants = gather_data(cities,'restaurants')```

```hotels = gather_data(cities,'hotelstravel')```

```nightlife = gather_data(cities,'nightlife')```

```beauty = gather_data(cities,'beautysvc')```

```shopping = gather_data(cities,'shopping')```

## 2. Collecting Affluence Data for each city

In [7]:
cities = pd.read_csv('../data/cities.csv')

In [8]:
cities_for_scrape = []
for i in list(cities['city,state']):
    cities_for_scrape.append(i.replace(',','').replace(' ','-'))
    
cities_for_scrape[11] = str('Boise-City-Idaho')
cities_for_scrape[41] = str('Nashville-Davidson-Tennessee')
cities_for_scrape[31] = str('New-York-New-York')

In [9]:
#Scraping city-data.com
#Written with assistance from Roy Kim - GA Denver
def get_income(list_of_cities_formatted,city_df):
    df = pd.DataFrame(columns = ['city','income','state','abbrev_state'])
    for c in list_of_cities_formatted:
        url = 'http://www.city-data.com/city/'+c+'.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        income = soup.find('section', {'class' : 'median-income'}).find('div', {'class' : 'hgraph'}).text
        income = income.split(':')[1].replace('$', '')
        income = re.findall('[^A-Z]*',income)[0].replace(',','').replace('$','')
        income = int(income)
        city = soup.find('section', {'class' : 'median-income'}).find('div', {'class' : 'hgraph'}).text
        city = city.split(':')[0]
        state = city_df['state'][list_of_cities_formatted.index(c)]
        abbrev_state = city_df['abbrev_state'][list_of_cities_formatted.index(c)]
        df = df.append({'city':city, 'income':income, 'state':state, 'abbrev_state':abbrev_state}, ignore_index=True)
    df.to_csv('../data/income.csv',index=False)
    return df

```income = get_income(cities_for_scrape,cities)```

## 3. Aggregating businesses by city

In [10]:
restaurants = pd.read_csv('../data/individual_restaurants.csv')
shopping = pd.read_csv('../data/individual_shopping.csv')
beauty = pd.read_csv('../data/individual_beautysvc.csv')
hotels = pd.read_csv('../data/individual_hotelstravel.csv')
nightlife = pd.read_csv('../data/individual_nightlife.csv')
cities = pd.read_csv('../data/cities.csv')
income = pd.read_csv('../data/income.csv')

In [11]:
def individual_to_agg(business_df,income_df,name_of_df):
    
    rows = []
    
    for row in business_df['state'].unique():
        row_dict = {}
        row_dict['abbrev_state'] = row
        row_dict['one_dollar'] = len(business_df[(business_df['price'] == '$') & (business_df['state'] == row)])
        row_dict['two_dollar'] = len(business_df[(business_df['price'] == '$$') & (business_df['state'] == row)])
        row_dict['three_dollar']  = len(business_df[(business_df['price'] == '$$$') & (business_df['state'] == row)])
        row_dict['four_dollar'] = len(business_df[(business_df['price'] == '$$$$') & (business_df['state'] == row)])
        row_dict['average_review_count_one_dollar'] = business_df[(business_df['state'] == row)&
                                                         (business_df['price'] == '$')]['review_count'].mean()
        row_dict['average_review_count_two_dollar'] = business_df[(business_df['state'] == row)&
                                                         (business_df['price'] == '$$')]['review_count'].mean()
        row_dict['average_review_count_three_dollar'] = business_df[(business_df['state'] == row)&
                                                         (business_df['price'] == '$$$')]['review_count'].mean()
        row_dict['average_review_count_four_dollar'] = business_df[(business_df['state'] == row)&
                                                         (business_df['price'] == '$$$$')]['review_count'].mean()
        row_dict['average_rating_one_dollar'] = business_df[(business_df['state'] == row) &
                                                  (business_df['price'] == '$')]['rating'].mean()
        row_dict['average_rating_two_dollar'] = business_df[(business_df['state'] == row) &
                                                  (business_df['price'] == '$$')]['rating'].mean()
        row_dict['average_rating_three_dollar'] = business_df[(business_df['state'] == row) &
                                                  (business_df['price'] == '$$$')]['rating'].mean()
        row_dict['average_rating_four_dollar'] = business_df[(business_df['state'] == row) &
                                                  (business_df['price'] == '$$$$')]['rating'].mean()
        row_dict['city_median_household_income'] = income_df[income_df['abbrev_state'] == row]['income'].mean()
        rows.append(row_dict)
    
    agg_df = pd.merge(pd.DataFrame(rows),income[['abbrev_state','city']],on='abbrev_state', how='left')
    agg_df.fillna(0,inplace=True)
    agg_df.to_csv(f"../data/aggregate_{name_of_df}.csv",index=False)
    return agg_df

In [12]:
restaurants_agg = individual_to_agg(restaurants,income,'restaurants')
shopping_agg = individual_to_agg(shopping,income,'shopping')
beauty_agg = individual_to_agg(beauty,income,'beauty')
hotels_agg = individual_to_agg(hotels,income,'hotels')
nightlife_agg = individual_to_agg(nightlife,income,'nightlife')